# Obce z lokalit

In [1]:
import numpy as np
import pandas as pd
lokality = pd.read_csv('eru_ceny_tepla_2018.csv')
lokality.shape

(3098, 33)

In [2]:
lokality.head()

,Unnamed: 0,lokalita,kraj,pod_uhli,pod_plyn,pod_bio,pod_olej,pod_ost,vykon,pocet_odberat,...,blok_kot_czk,blok_kot_gj,sek_rozv_czk,sek_rozv_gj,dps_czk,dps_gj,dom_kot_czk,dom_kot_gj,dod_cena,dod_mnozstvi
0,0,Abertamy,K,0.0,1.0,0.0,0.0,0.0,0.810,2,...,570.27,1139.0,0.0,0.0,0.00,0.0,0.00,0.0,570.27,1764.0
1,1,Adamov,B,0.0,1.0,0.0,0.0,0.0,2.203,1,...,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,224.94,22964.0
2,2,Adamov - Opletalova 38 a 22,B,0.0,1.0,0.0,0.0,0.0,0.460,2,...,0.00,0.0,0.0,0.0,0.00,0.0,662.43,1382.0,662.43,1382.0
3,3,Adamov - P. Jilemnického 18 (K 72),B,0.0,1.0,0.0,0.0,0.0,0.090,1,...,0.00,0.0,0.0,0.0,0.00,0.0,324.49,514.0,324.49,514.0
4,4,Adamov - teplovodní zdroj,B,0.0,1.0,0.0,0.0,0.0,9.000,37,...,0.00,0.0,0.0,0.0,540.92,26492.0,0.00,0.0,540.92,26492.0


In [3]:
lokality.columns

Index(['Unnamed: 0', 'lokalita', 'kraj', 'pod_uhli', 'pod_plyn', 'pod_bio',
       'pod_olej', 'pod_ost', 'vykon', 'pocet_odberat', 'pocet_mist',
       'nad_10_czk', 'nad_10_gj', 'pod_10_czk', 'pod_10_gj', 'cptv_czk',
       'cptv_gj', 'prim_rozv_czk', 'prim_rozv_gj', 'cvs_czk', 'cvs_gj',
       'cvs_voda_czk', 'cvs_voda_gj', 'blok_kot_czk', 'blok_kot_gj',
       'sek_rozv_czk', 'sek_rozv_gj', 'dps_czk', 'dps_gj', 'dom_kot_czk',
       'dom_kot_gj', 'dod_cena', 'dod_mnozstvi'],
      dtype='object')

In [4]:
# Odstranit sloupec s duplicitním indexem
lokality = lokality.iloc[:, 1:]

# Obce z lokalit
lokality['obec'] = lokality.lokalita.str.split('-', 1).str.get(0).str.strip()
lokality['obec'] = lokality['obec'].replace('[0-9]*/*,*', '', regex=True).str.strip()
lokality['obec'] = lokality['obec'].str.rstrip('I*V*X*').str.strip()
lokality.obec.value_counts().size

773

In [5]:
lokality['uhli'] = lokality['pod_uhli'] * lokality['dod_mnozstvi']
lokality['plyn'] = lokality['pod_plyn'] * lokality['dod_mnozstvi']
lokality['biomasa'] = lokality['pod_bio'] * lokality['dod_mnozstvi']
lokality['olej'] = lokality['pod_olej'] * lokality['dod_mnozstvi']
lokality['ostatni'] = lokality['pod_ost'] * lokality['dod_mnozstvi']

In [6]:
columns = ['obec', 'kraj', 'dod_mnozstvi', 'dod_cena',\
           'uhli', 'plyn', 'biomasa', 'olej', 'ostatni', \
           'vykon', 'pocet_odberat', 'pocet_mist',]
lokality = lokality[columns]

In [7]:
lokality[:10]

,obec,kraj,dod_mnozstvi,dod_cena,uhli,plyn,biomasa,olej,ostatni,vykon,pocet_odberat,pocet_mist
0,Abertamy,K,1764.0,570.270000,0.0,1764.0000,0.000,0.0000,0.0,0.810,2,1
1,Adamov,B,22964.0,224.940000,0.0,22964.0000,0.000,0.0000,0.0,2.203,1,1
2,Adamov,B,1382.0,662.430000,0.0,1382.0000,0.000,0.0000,0.0,0.460,2,2
3,Adamov,B,514.0,324.490000,0.0,514.0000,0.000,0.0000,0.0,0.090,1,1
4,Adamov,B,26492.0,540.920000,0.0,26492.0000,0.000,0.0000,0.0,9.000,37,28
5,Andělská Hora,T,917.0,558.900000,0.0,917.0000,0.000,0.0000,0.0,0.192,1,1
6,Arnoltice,M,1062.0,402.550000,1062.0,0.0000,0.000,0.0000,0.0,0.180,3,2
7,Aš,K,47004.0,566.416686,0.0,46674.9720,329.028,0.0000,0.0,15.650,58,81
8,Aš,K,1496.0,536.200000,0.0,1137.1096,0.000,358.8904,0.0,0.419,3,3
9,Babice,Z,5991.0,299.680000,0.0,5991.0000,0.000,0.0000,0.0,1.290,1,1


In [8]:
obce = pd.DataFrame()
grouped = lokality.groupby(by='obec')

In [9]:
obce['obec'] = grouped['obec'].first()
obce['kraj'] = grouped['kraj'].first()
obce['odberatelu'] = grouped['pocet_odberat'].sum()
obce['mist'] = grouped['pocet_mist'].sum()
obce['lokalit'] = grouped['pocet_odberat'].count()
obce['vykon'] = grouped['vykon'].sum()

In [10]:
obce['teplo_cena'] = grouped.apply(lambda x: np.average(x['dod_cena'], weights=x['dod_mnozstvi']))
obce['teplo_mnozstvi'] = grouped['dod_mnozstvi'].sum()

In [11]:
for col in ['uhli', 'plyn', 'biomasa', 'olej', 'ostatni']:
    obce[col] = grouped[col].sum()
    
obce['pod_uhli'] = obce['uhli']/obce['teplo_mnozstvi']
obce['pod_plyn'] = obce['plyn']/obce['teplo_mnozstvi']
obce['pod_bio'] = obce['biomasa']/obce['teplo_mnozstvi']
obce['pod_olej'] = obce['olej']/obce['teplo_mnozstvi']
obce['pod_ost'] = obce['ostatni']/obce['teplo_mnozstvi']

obce = obce.reset_index(drop=True)
obce[:10]

,obec,kraj,odberatelu,mist,lokalit,vykon,teplo_cena,teplo_mnozstvi,uhli,plyn,biomasa,olej,ostatni,pod_uhli,pod_plyn,pod_bio,pod_olej,pod_ost
0,Abertamy,K,2,1,1,0.810,570.270000,1764.0,0.0,1764.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000
1,Adamov,B,41,32,4,11.753,400.721314,51352.0,0.0,51352.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000
2,Andělská Hora,T,1,1,1,0.192,558.900000,917.0,0.0,917.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000
3,Arnoltice,M,3,2,1,0.180,402.550000,1062.0,1062.0,0.0000,0.0000,0.0000,0.0000,1.0,0.000000,0.000000,0.0000,0.000000
4,Aš,K,61,84,2,16.069,565.484642,48500.0,0.0,47812.0816,329.0280,358.8904,0.0000,0.0,0.985816,0.006784,0.0074,0.000000
5,Babice,Z,2,2,2,1.360,305.631533,6224.0,0.0,6224.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000
6,Batelov,J,2,2,2,0.252,505.413754,714.0,0.0,714.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000
7,Bechyně,C,38,58,1,7.300,555.840000,30596.0,0.0,125.4436,30470.5564,0.0000,0.0000,0.0,0.004100,0.995900,0.0000,0.000000
8,Benešov,S,58,160,5,68.933,574.074184,109257.0,0.0,106371.8771,0.0000,0.0000,2885.1229,0.0,0.973593,0.000000,0.0000,0.026407
9,Benešov nad Ploučnicí,U,14,18,2,3.657,607.226852,15424.0,0.0,15424.0000,0.0000,0.0000,0.0000,0.0,1.000000,0.000000,0.0000,0.000000


In [12]:
# Testing with Aš (two rows)
as_lokality = lokality[lokality.obec == 'Aš']
as_obce = obce[obce.obec == 'Aš']

In [13]:
as_obce.biomasa == as_lokality.biomasa.sum()

4    True
Name: biomasa, dtype: bool

In [14]:
as_obce.teplo_mnozstvi == as_lokality.dod_mnozstvi.sum()

4    True
Name: teplo_mnozstvi, dtype: bool

In [15]:
as_obce.teplo_cena == np.average(as_lokality.dod_cena, weights=as_lokality.dod_mnozstvi)

4    True
Name: teplo_cena, dtype: bool

In [16]:
lokality.dod_mnozstvi.sum() == obce.teplo_mnozstvi.sum()

True

In [17]:
np.average(lokality.dod_cena, weights=lokality.dod_mnozstvi) == np.average(obce.teplo_cena, weights=obce.teplo_mnozstvi)

False

In [18]:
np.average(lokality.dod_cena, weights=lokality.dod_mnozstvi)

421.0602087860089

In [19]:
np.average(obce.teplo_cena, weights=obce.teplo_mnozstvi)

421.06020878600896

In [20]:
obce.to_csv('obce_2018.csv', na_rep='n/a', index=False)